[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/SPE_GeoHackathon_2025/blob/dev/S1_M2_ChatAgent.ipynb)

***
# Session 01 // Module 03: Agent Interfaces

### What is an agent?

An agent is an LLM-driven decision-maker that can plan steps and choose tools to reach a goal. Instead of a fixed, linear chain, an agent decides what to do next at runtime.

- Core pieces:
  - LLM: the “brain” that reasons about the task.
  - Prompt/policy: instructions + scratchpad that guide the LLM’s decisions.
  - Tools: callable functions/APIs (e.g., web search, calculators, vector stores).
  - Memory (optional): prior turns or facts that persist across steps.
  - Executor/loop: runs the think → act (tool) → observe cycle until the agent decides it’s done.

- When to use:
  - Multi-step tasks with uncertainty or branching.
  - When external tools/data are needed (RAG, code, search, math).
  - When you want the model to decide the next best action dynamically.

- Chain vs Agent:
  - Chain: predefined flow (prompt → model → parser).
  - Agent: dynamic flow chosen by the model at runtime.

In this notebook, we build a memory-enabled chat workflow (a building block for agents). You can later add tools and wrap with an Agent executor to turn it into a full tool-using agent.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Environment setup
!pip -q install langchain langchain-core langchain-community langchain-huggingface torch gradio
!pip -q install bitsandbytes==0.46.0 transformers==4.48.3 

In [ ]:
# Hugging Face API token
# Retrieving the token is required to get access to HF hub
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

## 7. Designing a simple agent class

We wrap the chain and memory plumbing into a class to:
- Encapsulate system prompts and chains.
- Offer simple methods (chat, clear_memory, get_history, create_new_session).
- Swap models without changing the interface.

Class design tips:
- Keep system prompts centralized and editable.
- Return clean strings to the UI layer.
- Add basic try/except for robust demos.

```python
response = agent.chat("What role does well logging play?")
```

In [ ]:
from typing import Dict, Any
import uuid

class ModernGeoscienceChatAgent:
    def __init__(self, chat_model):
        self.chat_model = chat_model
        self.store = {}
        
        # Enhanced system prompt
        self.system_prompt = """
You are Dr. GeoBot, a friendly and knowledgeable geoscience expert specializing in:
- Geophysics and seismic interpretation
- Petroleum geology and reservoir engineering  
- Well logging and formation evaluation
- Hydrocarbon exploration and production
- Geomechanics and drilling engineering

Guidelines:
- Provide accurate, helpful answers about geoscience topics
- Use technical terms but explain them when needed
- Be conversational and engaging
- Keep responses focused and informative
- If unsure, acknowledge limitations honestly
- Reference previous conversation when relevant
"""
        
        # Create prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}")
        ])
        
        # Create chain
        self.chain = self.prompt | self.chat_model | StrOutputParser()
        
        # Create conversational chain with memory
        self.conversational_chain = RunnableWithMessageHistory(
            self.chain,
            self.get_session_history,
            input_messages_key="question",
            history_messages_key="history",
        )
    
    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]
    
    def chat(self, question: str, session_id: str = "default") -> str:
        """Process a question and return a response"""
        try:
            config = {"configurable": {"session_id": session_id}}
            response = self.conversational_chain.invoke(
                {"question": question},
                config=config
            )
            return response.strip()
        except Exception as e:
            return f"I apologize, but I encountered an error: {str(e)}"
    
    def clear_memory(self, session_id: str = "default"):
        """Clear conversation history for a session"""
        if session_id in self.store:
            self.store[session_id].clear()
    
    def get_history(self, session_id: str = "default") -> list:
        """Get conversation history for a session"""
        if session_id in self.store:
            return self.store[session_id].messages
        return []
    
    def create_new_session(self) -> str:
        """Create a new conversation session"""
        return str(uuid.uuid4())

# Create the modern chat agent
chat_agent = ModernGeoscienceChatAgent(chat_model)
print("Modern GeoscienceChatAgent created successfully!")

In [ ]:
# Test the modern chat agent
print("=== Testing Modern GeoscienceChatAgent ===")

# Test conversation
questions = [
    "Hello! Can you explain what you specialize in?",
    "What is the difference between conventional and unconventional reservoirs?",
    "How do geophysicists use seismic data to find oil?",
    "What role does well logging play in this process?"
]

session_id = chat_agent.create_new_session()
print(f"Created session: {session_id[:8]}...\n")

for i, question in enumerate(questions, 1):
    print(f"{i}. Human: {question}")
    response = chat_agent.chat(question, session_id)
    print(f"   Dr. GeoBot: {response}")
    print("-" * 100)

## 8. Building a Gradio UI

We expose the agent through a browser-based chat.

Core components:
- gr.Blocks: Page/layout container.
- gr.Chatbot: Conversation pane (list of (user, bot) tuples).
- gr.Textbox: Input field for user messages.
- gr.Button: Send, Clear, Load examples.
- Events: .submit and .click wire UI to Python callbacks.

UX tips:
- Keep answers concise; use memory to reduce repetition.
- Provide example prompts to guide first-time users.
- Add a “Clear chat” to reset session/memory.

```python
def respond(message, history):
    reply = agent.chat(message, session_id)
    history.append((message, reply))
    return "", history

msg.submit(respond, [msg, chatbot], [msg, chatbot])
send_btn.click(respond, [msg, chatbot], [msg, chatbot])
clear_btn.click(lambda: agent.clear_memory(session_id) or [], outputs=chatbot)
```

In [ ]:
import gradio as gr
from typing import List, Tuple

# Create a new chat agent for the interface
gradio_agent = ModernGeoscienceChatAgent(chat_model)

# Global session management
current_session = gradio_agent.create_new_session()

def respond(message: str, history: List[Tuple[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Process user message and return bot response
    """
    global current_session
    
    if not message.strip():
        return "", history
    
    # Get response from agent
    bot_response = gradio_agent.chat(message, current_session)
    
    # Add to chat history
    history.append((message, bot_response))
    
    return "", history

def clear_conversation() -> List[Tuple[str, str]]:
    """
    Clear conversation history and start new session
    """
    global current_session
    gradio_agent.clear_memory(current_session)
    current_session = gradio_agent.create_new_session()
    return []

def load_example(example: str) -> str:
    """
    Load example question into the textbox
    """
    return example

# Create modern Gradio interface
with gr.Blocks(
    title="Dr. GeoBot - Advanced Geoscience Chat Assistant",
    theme=gr.themes.Soft()
) as demo:
    
    gr.Markdown("""
    # 🌍 Dr. GeoBot - Your Advanced Geoscience Expert
    
    I'm an AI geoscience expert powered by modern LangChain. Ask me about:
    
    | **Geophysics** | **Petroleum Engineering** | **Well Logging** |
    |---|---|---|
    | Seismic interpretation | Reservoir characterization | Formation evaluation |
    | Gravity & magnetics | Hydrocarbon systems | Petrophysics |
    | Electromagnetics | Production optimization | Log analysis |
    
    💡 *I remember our conversation, so feel free to ask follow-up questions!*
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                value=[],
                height=500,
                show_label=False,
                bubble_full_width=False
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Ask me about geoscience topics...",
                    show_label=False,
                    scale=4,
                    container=False
                )
                send_btn = gr.Button("Send 📤", scale=1, variant="primary")
            
            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")
                
        with gr.Column(scale=1):
            gr.Markdown("### 💡 Example Questions")
            
            example_questions = [
                "What is seismic inversion?",
                "Explain porosity vs permeability",
                "How do P-waves and S-waves differ?",
                "What is reservoir characterization?",
                "How does well logging work?",
                "What are the challenges in unconventional reservoirs?"
            ]
            
            for question in example_questions:
                example_btn = gr.Button(
                    question,
                    variant="secondary",
                    size="sm"
                )
                example_btn.click(
                    load_example,
                    inputs=[gr.State(question)],
                    outputs=msg
                )
    
    # Event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(clear_conversation, outputs=chatbot)

# Launch the interface
print("Launching modern Gradio interface...")
demo.launch(share=True, show_error=True)

## 11. Quick reference

- Keep prompts short and specific; push style guidance to the system message.
- temperature: 0.2–0.7 for helpful chat; higher for creativity.
- max_new_tokens: 100–200 for chat; lower = faster.
- Use LCEL to compose; add memory with RunnableWithMessageHistory.
- Test locally with tiny models; switch to Colab GPU for 7–8B.

***